The goal of this notebook is to populate the VGG framework with pre trained weights from a model that achieved very high performance using transfer learning to learn from the fashion mnist data set, run polo instagram and ebay related images through the network in order to obtain the ebmeddings that are normally used as input to the softmax layer, perform a variety of analysis to measure the similarity or differences between the two sources as well as within the sources. Specifically, it would be very interesting to analyze the sub groups within the instagram photos to see if stylistc patterns emerge.

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
#from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [10]:
from zipfile import ZipFile
from pathlib import Path

# Extract file names from zip files, convert to dataframe and remove directories

In [53]:
def zipToDf(zip_file):
    #collect all file names contained in zip file
    with ZipFile(zip_file, 'r') as f:
        file_names = f.namelist()
    
    df = pd.DataFrame({'path':file_names})
    
    #remove directories
    df = df.loc[df.path.str.slice(-1) != '/']
    
    #remove 
    
    return df

## Read in insta images from #vintagepolo

In [54]:
insta_zip_file = '/Users/chrisjonas/projects/GitHub/insta_images/vintagepolo.zip'

insta_df = zipToDf(insta_zip_file)

In [81]:
insta_df

,path
1,vintagepolo/10597301_261438900718846_171175809...
2,vintagepolo/10693363_1483677961885378_62745728...
3,vintagepolo/10787815_719888748126871_150763782...
4,vintagepolo/10787839_750596838364426_566095108...
5,vintagepolo/10809568_311430665719110_715746864...
6,vintagepolo/10810002_1501858423425367_13025514...
7,vintagepolo/16908522_1867338436874871_35605429...
8,vintagepolo/18580439_1511242428948505_51627678...
9,vintagepolo/21984735_519834648354453_700461780...
10,vintagepolo/22221332_155533781706482_647063493...


In [57]:
len(insta_df)

24007

## Read in eBay images from polo BIN search

In [47]:
ebay_zip_file = '/Users/chrisjonas/projects/GitHub/ebay_images/images/polo.zip'

ebay_df = zipToDf(ebay_zip_file)

In [58]:
len(ebay_df)

4808

## Read in eBay photos from polo search

In [ ]:
zip_file = '/Users/chrisjonas/projects/GitHub/insta_images/vintagepolo.zip'

with ZipFile(zip_file, 'r') as f:
    file_names = f.namelist()

In [59]:
# Read csv data files
train_data = pd.read_csv('~/Downloads/fashion-mnist_test.csv.zip',compression='zip')
#test_data = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

In [71]:
#splitting off label from image

train_data.shape #(60,000*785)
#test_data.shape #(10000,785)
train_X= np.array(train_data.iloc[:,1:])
#test_X= np.array(test_data.iloc[:,1:])
train_Y= np.array (train_data.iloc[:,0]) # (60000,)
#test_Y = np.array(test_data.iloc[:,0]) #(10000,)

In [4]:
train_data.head()

In [80]:
([train_X] * 3)

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 3, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 3, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 1, 3, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [5]:
train_X.shape, test_X.shape

In [6]:
classes = np.unique(train_Y)
num_classes = len(classes)
num_classes

In [7]:
# Convert the images into 3 channels
train_X=np.dstack([train_X] * 3)
test_X=np.dstack([test_X]*3)
train_X.shape,test_X.shape

In [8]:
# Reshape images as per the tensor format required by tensorflow
train_X = train_X.reshape(-1, 28,28,3)
test_X= test_X.reshape (-1,28,28,3)
train_X.shape,test_X.shape

In [9]:
# Resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
train_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in train_X])
test_X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in test_X])
#train_x = preprocess_input(x)
train_X.shape, test_X.shape

In [10]:
# Normalise the data and change data type
train_X = train_X / 255.
test_X = test_X / 255.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

In [11]:
# Converting Labels to one hot encoded format
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

In [12]:
# Splitting train data as train and validation data
train_X,valid_X,train_label,valid_label = train_test_split(train_X,
                                                           train_Y_one_hot,
                                                           test_size=0.2,
                                                           random_state=13
                                                           )

In [13]:
# Finally check the data size whether it is as per tensorflow and VGG16 requirement
train_X.shape,valid_X.shape,train_label.shape,valid_label.shape

In [14]:
# Define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [15]:
# Preprocessing the input 
train_X = preprocess_input(train_X)
valid_X = preprocess_input(valid_X)
test_X  = preprocess_input (test_X)

In [16]:
#  Create base model of VGG16
conv_base = VGG16(weights='../input/keras-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()

In [17]:
# Extracting features
train_features = conv_base.predict(np.array(train_X), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(test_X), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(valid_X), batch_size=BATCH_SIZE, verbose=1)
#for layer in conv_base.layers:
#    layer.trainable = False

In [18]:
# 6.1 Saving the features so that they can be used for future
np.savez("train_features", train_features, train_label)
np.savez("test_features", test_features, test_Y)
np.savez("val_features", val_features, valid_label)

In [19]:
# Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

In [20]:
# Flatten extracted features
train_features_flat = np.reshape(train_features, (48000, 1*1*512))
test_features_flat = np.reshape(test_features, (10000, 1*1*512))
val_features_flat = np.reshape(val_features, (12000, 1*1*512))

In [31]:
from keras import models
from keras.models import Model
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers.advanced_activations import LeakyReLU

In [32]:
# 7.0 Define the densely connected classifier followed by leakyrelu layer and finally dense layer for the number of classes
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 100

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(num_classes, activation='softmax'))

In [33]:
# Compile the model.
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(),
  # optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc'])

In [34]:
# Incorporating reduced learning and early stopping for callback
reduce_learning = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    mode='auto',
    epsilon=0.0001,
    cooldown=2,
    min_lr=0)

eary_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=1,
    mode='auto')

callbacks = [reduce_learning, eary_stopping]

In [35]:
# Train the the model
history = model.fit(
    train_features_flat,
    train_label,
    epochs=NB_EPOCHS,
    validation_data=(val_features_flat, valid_label),
    callbacks=callbacks
)

In [36]:
# plot the loss and accuracy

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')

plt.legend()

plt.show()


References
https://www.kaggle.com/crawford/diagnose-lung-disease-with-vgg16
https://www.programcreek.com/python/example/92213/keras.applications.vgg16.VGG16
http://www.socouldanyone.com/2013/03/converting-grayscale-to-rgb-with-numpy.html